# Emulating realistically bad-for-cosmology SN Ia samples from PLAsTiCC data

In [ ]:
import gzip
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle as pkl
# import proclam

rando = 42

In [ ]:
datapath = '/media/RESSPECT/data/PLAsTiCC/PLAsTiCC_zenodo/'

classes we care about

| `true_target`=`type` | `code` |
| -------------------- | ------ |
| 90 | SNIa |
| 67 | SNIa-91bg |
| 52 | SNIax |
| 42 | SNII |
| 62 | SNIbc |
| 95 | SLSN-I |
| 88 | AGN |

In [ ]:
maybe_sn_classes = {90: 'SNIa', 
                    67: 'SNIa-91bg', 
                    52: 'SNIax', 
                    42: 'SNII', 
                    62: 'SNIbc', 
                    95: 'SLSN-I', 
                    88: 'AGN'}
maybe_sn_classes[64] = 'KN'
maybe_sn_classes[15] = 'TDE'

## gather all DDF lightcurves

only do it once

from validation set

In [ ]:
# with open(datapath+'plasticc_train_metadata.csv', 'rb') as filename:
#     train = pd.read_csv(filename)
    
# train_in_ddf = train[train['ddf_bool']==1]
# # print(str(len(train_in_ddf))+' in DDF of '+str(len(train))+' total training set') 
# print(train_in_ddf.groupby('true_target').count())

from test set

In [ ]:
# with gzip.open(datapath+'plasticc_test_metadata.csv.gz', 'rb') as filename:
#     test = pd.read_csv(filename)

# test_in_ddf = test[test['ddf_bool']==1]
# # print(str(len(test_in_ddf))+' in DDF of '+str(len(test))+' total test set') 
# print(test_in_ddf.groupby('true_target').count())

put them together and save critical info for later

In [ ]:
# maybe_sn_train = train_in_ddf.loc[train_in_ddf['true_target'].isin(maybe_sn_classes.keys())]
# maybe_sn_test = test_in_ddf.loc[test_in_ddf['true_target'].isin(maybe_sn_classes.keys())]
# all_maybe_sn = pd.concat((maybe_sn_train, maybe_sn_test))[['object_id', 'hostgal_specz', 'true_target']].set_index('object_id')
# all_maybe_sn.to_csv('ddf_only.csv')

## subsample the classes to make new samples

go +/- 25% from fiducial contamination rates from a (forthcoming) confusion matrix at `/media/RESSPECT/data/PLAsTiCC/for_metrics/confusion_matrices`

In [ ]:
all_maybe_sn = pd.read_csv('ddf_only.csv')#, index_col='object_id')

In [ ]:
all_maybe_sn.columns

In [ ]:
savecounts = all_maybe_sn.groupby('true_target').count()['object_id'].to_numpy()
print(savecounts)

In [ ]:
with open('/media/RESSPECT/data/PLAsTiCC/for_metrics/confusion_matrices/confusion_matrix.npy', 'rb') as confmat:
    cm = np.load(confmat)
    
print(cm)

These are just test set lightcurves for classes (67, 88, 42(minus 7?), 90(minus 11?), 52, 62, 64, 95, 15)
                    
So take out -1 and -3 row/col to match the classes being considered here?

In [ ]:
cm_classes = [67, 88, 42, 90, 52, 62, 64, 95, 15]
cm_indices = {}
for classid in maybe_sn_classes.keys():
    cm_indices[classid] = cm_classes.index(classid)
cm_indices

get sample ids for a given row

In [ ]:
def gen_samp_ids(cm, cm_indices, samp_key, savepath=None, rando=rando):
    cm_row = cm.T[cm_indices[samp_key]]
#     print((cm_row, np.sum(cm_row)))
    out_ids = []
    for typeid in cm_indices.keys():
        n_to_sample = cm_row[cm_indices[typeid]]
        print(n_to_sample)
        matches = all_maybe_sn['object_id'][all_maybe_sn['true_target'] == typeid].sample(n=n_to_sample)#, random_state=rando)
        out_ids += matches.to_list()
    if savepath:
        with open(savepath+'.pkl', 'wb') as tosave:
            pkl.dump(out_ids, tosave)
    return(out_ids)

In [ ]:
type(n_to_sample)

In [ ]:
fiducial = gen_samp_ids(cm, cm_indices, 90, savepath='/media/RESSPECT/data/PLAsTiCC/for_metrics/fiducial_ids')
# print(len(fiducial))

In [ ]:
cm_perfect = np.diag(cm) * np.identity(len(cm_indices.keys()))
perfect = gen_samp_ids(cm_perfect, cm_indices, 90, savepath='/media/RESSPECT/data/PLAsTiCC/for_metrics/perfect_ids')

In [ ]:
def perturb_subsamp(cm, cm_indices, samp_key, epsilon):
    cm_row = cm.T[cm_indices[samp_key]]
    for typeid in cm_indices.keys():
        if typeid != samp_key:
            

## 

start with purity and efficiency for binary classification situation

In [ ]:
tot_in_test = np.sum(cm, axis=1)
print(tot_in_test)

tp_in_test = np.diag(cm)
tpr_in_test = tp_in_test / tot_in_test
# print((tp_in_test, tpr_in_test))
efficiency = tpr_in_test
print(efficiency)

contamination_raw = np.sum(cm, axis=0) - tp_in_test
contamination_rate = contamination_raw / tot_in_test
# print((contamination_raw, contamination_rate))
purity = tp_in_test / (tp_in_test + contamination_raw)
print(purity)

## save outputs

as `id,redshift,type,code,orig_sample=test,queryable=True`